<a href="https://colab.research.google.com/github/xiao-yucheng0625/ML-Progression-Journal/blob/main/Goal_Backward_CoT_%E5%A4%A7%E5%AD%B8%E7%94%9F%E5%AD%B8%E7%BF%92%E6%95%88%E7%8E%87%E6%8F%90%E5%8D%87%E8%A8%88%E7%95%AB.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 大學生學習效率提升計劃

本任務結合了「Goal-Backward Chain of Thought (CoT)」與「Reflection and Revision」方法，目的是讓AI能夠：

* 從最終學習目標出發，反向推導每一個子目標與具體行動

* 撰寫可執行的詳細計劃

* 進行主動反思與優化

整個過程模擬人類的長期規劃能力與自我改進能力，並能夠反覆優化策略，確保計畫實際可行且高效。



# Goal-Backward CoT

## 第一階段-思考階段
根據「提升大學生學習效率」的主題，先設定一個明確且具挑戰性的最終目標（例如：半年內GPA達4.0），依照逆推原則，逐層推導：
* 若要達成這個最終目標，需要什麼子目標？

* 每個子目標又需要哪些更基礎的具體行動？

形成一條「目標 ➔ 子目標 ➔ 具體行動」的完整路徑圖
## 第二階段-產文階段

根據逆推出來的結構，撰寫一份清楚、具體、可執行的初步學習計畫，初稿內容結構至少包含：

1. 最終目標
2. 中間子目標（分層列出）
3. 每個子目標對應的具體行動（列點式描述）
4. 預期每階段的達成時間
5. 可能的挑戰與應對方案



## 1. 讀入金鑰
請依你使用的服務，決定要讀入的金鑰。

In [1]:
import os
from google.colab import userdata

In [2]:
#【使用 Mistral】
# api_key = userdata.get('Mistral')
# os.environ['MISTRAL']=api_key
# provider = "mistral"
# model = "ministral-8b-latest"

#【使用 OpenAI】
# api_key = userdata.get('OpenAI')
# os.environ['OPENAI_API_KEY']=api_key
# provider = "openai"
# model = "gpt-4o"

#【使用 Groq】
api_key = userdata.get('Groq')
os.environ['GROQ_API_KEY']=api_key
provider = "groq"
model = "llama3-70b-8192"

In [3]:
!pip install aisuite[all]

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 863.9/863.9 kB 13.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 88.2/88.2 kB 6.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 259.5/259.5 kB 18.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 103.5/103.5 kB 7.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.4/76.4 kB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.2/41.2 kB 3.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 64.6 MB/s eta 0:00:00
  Attempting uninstall: httpx
    Found existing installation: httpx 0.28.1
    Uninstalling httpx-0.28.1:
      Successfully uninstalled httpx-0.28.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-genai 1.11.0 requires httpx<1.0.0,>=0.28.1, but you have httpx 0.27.2 which is incompatible.


## 2. 載入 AI Suite
讓我們方便切換多家AI服務商（provider）API的套件，它幫你把常見的大模型 API（像 OpenAI、Groq、Mistral、Anthropic、Perplexity）統一整理成一個好用的介面，這樣你在寫程式時，不用記每家公司的 API 怎麼接，只要學一種寫法就可以了。

In [4]:
import aisuite as ai

* 設定 planner 使用的 provider 與 model 名稱
* 設定 writer 使用的 provider 與 model 名稱

In [5]:
provider_planner = "groq"
model_planner="llama3-70b-8192"

provider_writer = "groq"
model_writer = "llama3-70b-8192"

定義一個 reply 函式，接收一個 system 訊息作為參數

In [6]:
def reply(system="請用台灣習慣的中文回覆。",
          prompt="Hi",
          provider="groq",
          model="llama3-70b-8192"
          ):

    client = ai.Client()

    messages = [
        {"role": "system", "content": system},
        {"role": "user", "content": prompt}
    ]


    response = client.chat.completions.create(model=f"{provider}:{model}", messages=messages)

    return response.choices[0].message.content

## 3. 打造 CoT 二階段-思考以及產文

In [7]:
system_planner = "請用台灣習慣的中文回應: 你是一位擅長目標導向規劃的顧問，根據使用者給定的問題，從最終目標出發，反向推導中間子目標與具體行動。"
system_writer = "請用台灣習慣的中文回應: 你是一位善於撰寫具體可執行計畫的助理，根據已完成的目標-子目標-行動樹，撰寫一份結構完整、內容具體且具有可行性的學習提升計畫。"

In [8]:
def learning_plan(prompt):
    # Step 1: CoT - 思考目標導向逆推的方式
    planning_prompt = f"使用者說：{prompt}。請幫我想從最終目標反推每一層子目標，直到到達可以當天執行的具體行動。"
    learning_goals = reply(system_planner, planning_prompt,
                          provider = provider_planner,
                          model = model_planner
                          )

    # Step 2: 撰寫出版詳細計畫
    generation_prompt = f"這是我想到的學習目標：\n{learning_goals}\n\n請根據學習目標撰寫一份詳細可執行的學習計畫。"
    first_plan = reply(system_writer, generation_prompt,
                       provider = provider_writer,
                       model = model_writer
                       )

    return learning_goals, first_plan

# Reflection

## 第一階段-評估並建議
仔細閱讀初版學習計畫，並且評估初稿
* 優點（清楚、可行、完整性）

* 缺點（模糊、不可操作、疏漏）

* 改進建議（具體指出如何補強）

## 第二階段-修改階段
修正並優化原本的學習計畫，補強目標清晰度、行動細節與時間規劃，使最終版計畫更具備：

* 明確性

* 邏輯性

* 可執行性

* 持續追蹤性

## 4. 設定審查員以及改寫員

In [9]:
provider_reviewer = "groq"
model_reviewer = "qwen-qwq-32b"

provider_rewriter = "groq"
model_rewriter = "qwen-qwq-32b"

In [10]:
system_reviewer = "你是一位學習計畫顧問，請仔細評估以下學習計畫的優點與缺點，並提出具體、可行的改進建議。請用台灣常用的中文，簡潔明確。"
system_rewriter = "你是一位學習輔導助理，請根據建議，將學習計畫改寫得更具體、可執行，並補充細節與時間安排。請用自然流暢的台灣中文表達。"

In [11]:
def reflect_plan(first_plan):
    # Step 1: Reviewer 給建議
    suggestion = reply(system_reviewer, first_plan,
                       provider=provider_reviewer,
                       model=model_reviewer
                       )

    # Step 3: Writer 再寫一次（根據建議）
    second_prompt = f"這是我剛剛寫的計畫：\n{first_plan}\n\n這是修改建議：\n{suggestion}\n\n請根據這些建議，幫我改得更具體、更可執行。請用台灣習慣的中文, 並且只要輸出改好的文章就可以了。"
    second_plan = reply(system_rewriter, second_prompt,
                provider=provider_rewriter,
                model=model_rewriter
                )

    return suggestion, second_plan

# 5. 用 Gradio 打造你的對話機器人 Web App!

In [12]:
!pip install gradio

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.0/54.0 MB 17.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 322.6/322.6 kB 21.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 95.2/95.2 kB 7.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.5/11.5 MB 122.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.0/72.0 kB 5.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.5/62.5 kB 4.6 MB/s eta 0:00:00


In [13]:
import gradio as gr

In [14]:
def full_process(prompt):
    # Step 1: 根據使用者輸入，進行 Goal-Backward 思考
    learning_goals, first_plan = learning_plan(prompt)

    # Step 2: 根據 first_plan，進行反思與修正
    suggestion, second_plan = reflect_plan(first_plan)

    # Step 3: 回傳所有結果
    return learning_goals, first_plan, suggestion, second_plan

In [15]:
with gr.Blocks() as demo:
    gr.Markdown("### 🤖 學習計畫反思幫手（Reflection Agent）")
    user_input = gr.Textbox(label="請輸入你今天想分享的內容")
    btn = gr.Button("生成計畫 & 修正建議")

    with gr.Row():
        out0 = gr.Textbox(label="🎯 目標導向思考 (model_planner)")
        out1 = gr.Textbox(label="🌟 第一版計畫 (model_writer)")
        out2 = gr.Textbox(label="🧐 修改建議 (model_reviewer)")
        out3 = gr.Textbox(label="✨ 第二版計畫 (model_rewriter 改寫)")

    btn.click(full_process, inputs=[user_input], outputs=[out0, out1, out2, out3])

In [ ]:
demo.launch(share=True, debug=True)

Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
* Running on public URL: https://17d1d357c7df91c3b3.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
